$$\large{\mathbf{Instituto\ Superior\ de\ Engenharia\ de\ Lisboa}}$$

$$\large{\mathrm{Licenciatura\ em\ Engenharia\ Informática\ e\ Multimédia}}$$

$$\Large{\mathbf{Aprendizagem \ Automatica}}$$

$$\normalsize{\mathbf{Trabalho \ Laboratorial-\ Rate \ Beer \ Dataset}}$$

$$\normalsize{\mathbf{Pedro\ Silva\ (48965)\\}}$$


<a id='section0'></a>
# Índice
<hr style="height:3px"/>

1. [Introdução](#section1)
2. [Construção do vocabulário](#section2)
    1. [Inicialização](#section2.1)
    2. [Modelo Bag of Words (BoW)](#section2.2)
3. [Classificação Binária](#section3)
    1. [Treino](#section3.1)
    2. [Teste](#section3.2)
4. [Classificação Multi-Classe](#section4)
    1. [Treino](#section4.1)
    2. [Teste](#section4.2)
5. [PCA](#section5)
6. [Conclusões](#section6)

# 1. Introdução <a id='section1'></a>
<hr style="height:3px"/>
Em termos globais, o que se pretende e determinar a qualidade de uma cerveja baseado no que foi escrito sobre a mesma. Neste contexto surgem duas tarefas de classificação a ser implementadas:

## Classificação Binária:
Nesta tarefa, pretende-se saber se o crítico considera a cerveja muito boa ou muito má, baseado no que escreveu. Considere que uma cerveja e considerada muito boa quando obteve uma pontuação global (campo overall) de 9 ou mais valores. Considere ainda que uma cerveja e considerada muito má quando obteve uma pontuação global de 2 ou menos valores.

## Classificação Multi-Classe: 
Prever a pontuação de três aspetos das críticas (smell, taste e overall). Neste ponto,
treine e avalie os classificadores com os dados de treino e verifique se as estimativas do
desempenho condizem com os resultados obtidos no conjunto de teste.



# 2. Construção do vocabulário <a id='section2'></a>

## 2.1. Inicialização <a id='section2.1'></a>
<hr style="height:3px"/>

Vamos realizar os imports e inicializar os dados que vamos utilizar neste caso o ficheiro: 'rateBeer75Ktrain.p'. Vamos extrair também o valor da review ao tirar o valor antes da '/'. Criamos também uma função para converter uma string em tfidf dado um modelo.

In [ ]:
import pickle
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import strip_accents_ascii
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
import time
from sklearn.decomposition import TruncatedSVD

# Carregar os dados
dados_treino = pickle.load(open('rateBeer75Ktrain.p', 'rb'))

# Extrair características (avaliações) e rótulos (classificações gerais)
avaliacoes = [dados_treino[chave]['review'] for chave in dados_treino]

# Extrair a parte numérica antes de '/' do campo 'overall'
overall = [int(re.search(r'\d+', dados_treino[chave]['overall']).group()) for chave in dados_treino]

# Dividir os dados em conjuntos de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(avaliacoes, overall, test_size=0.4, random_state=42)

# Função para converter uma string de texto ou lista de strings na representação TF-IDF
def converte_texto_para_tfidf(modelo, texto):
    # Se o texto for uma lista de strings, junte as strings em um único texto
    if isinstance(texto, list):
        texto = ' '.join(texto)
    
    # Use o transform para converter o texto em representação TF-IDF
    representacao_tfidf = modelo.named_steps['tfidf'].transform([texto])
    
    return representacao_tfidf


## 2.2. Modelo Bag of Words (BoW)<a id='section2.2'></a>
<hr style="height:3px"/>

Uma das formas de representar texto é o modelo “Bag of Words” (i.e. saco de palavras). Nesta representação a forma, estrutura do texto é descartada bem como a ordem das palavras e é só tido em conta o número de ocorrências de cada palavra em cada documento do corpus. O resultado final é uma matriz denominada “documento-termo” (do inglês _document-term matrix_) com dimensão Nxd, onde N é número de documentos no corpus e d é o número de palavras do vocabulário. BoW é uma técnica não supervisionada de representar um texto por um vetor numérico.

A representação BoW consiste nos seguintes passos:

1. **Tokenization**
   Este processo consiste em dividir cada documento em palavras (ou _tokens_), por exemplo separando as palavras nos textos através dos caracteres de espaço ou pontuação.
2. **Construção do Vocabulário:**
   Construir um vocabulário constituído por todas ou por um sub-conjunto das palavras presentes no corpus.
3. **Codificação:**
   - Contar o número de vezes que cada palavra do vocabulário aparece em cada documento.
   - Representar cada documento por um vetor de d dimensões, uma por cada palavra no vocabulário, com valores proporcionais ao número de ocorrências dessa palavra no documento. (Nota: estes vetores terão a maior parte dos seus coeficientes a zero)

## Questões Práticas:

1. Cada texto é representado por um vetor de dimensão igual ao número de palavras no vocabulário.

2. Devido à diversidade de termos, palavras, interjeições, etc., presentes na maioria dos idiomas, se não houver nenhum pré-processamento dos textos, o vocabulário pode ter vários milhares de palavras.

3. É por isso aconselhável, antes de fazer a representação BoW, processar os documentos da base de dados de forma a reduzir a dimensão do vocabulário. Existem várias técnicas, tais como considerar só palavras que tenham um número de ocorrências superior a um dado limiar, converter palavras semelhantes numa única palavra, etc.

4. O processo de limpeza tem que ser o mesmo para todos os documentos (documentos de treino e de teste, e outros documentos nunca classificados), e deve ser aplicado antes de obter a representação BoW.

## Limpeza do texto

Existem muitos caracteres ou sequências de caracteres, como mudanças de linha em HTML `<br />`, que não contribuem para discriminar entre boas e más críticas, mas podem prejudicar bastante o desempenho dos classificadores. Existem alguns comandos simples que ajudam a limpar o vocabulário.

As classes CountVectorizer e TfidfVectorizer também realizam uma limpeza dos dados. Essas classes extraem apenas palavras com comprimento maior que dois caracteres e convertem os caracteres alfabéticos para minúsculas.

Além disso, existe a possibilidade de extrair apenas as palavras que aparecem em mais do que um número pré-definido de documentos usando o parâmetro `min_df` (frequência mínima do documento). Por exemplo, o seguinte comando extrai apenas palavras que aparecem em 5 ou mais documentos:

Além disso, é possível especificar qual expressão regular é usada no processo de tokenização. Por padrão, a expressão regular é `r"\b\w\w+\b"`. Isso significa que serão extraídas sequências de caracteres compostas por 2 ou mais letras ou números (`\w`) e que estão separadas por caracteres de pontuação ou espaços (`\b`).


## Stop Words

Outra maneira de reduzir o tamanho do vocabulário é eliminar palavras através de uma lista de "stop words". Stop words são palavras que ocorrem frequentemente em uma dada língua (cada idioma tem seu conjunto específico de stop words). O scikit-learn contém uma lista de stop words em inglês no módulo `feature_extraction.text`.

A remoção das stop words não tem uma contribuição significativa para o melhoramento da caracterização dos documentos. Pode-se treinar modelos com vocabulários com e sem stop words e verificar se há melhoria no desempenho. Em certas situações, como é o caso de n-gramas, a remoção pode até ser prejudicial. Existem outros métodos mais eficazes para reduzir a dimensão do vocabulário.

## Stemming

Essa técnica consiste no processo de transformar uma palavra em sua raiz, o que permite mapear palavras semelhantes em uma única palavra. Por exemplo, palavras como "studies", "studying", "studied" seriam mapeadas para "studi".

O primeiro algoritmo de stemming foi desenvolvido por Martin F. Porter e ficou conhecido como Porter Stemmer. Este algoritmo está disponível no módulo Natural Language Toolkit (nltk) em Python, juntamente com outros algoritmos de stemming, como os casos do Snowball e Lancaster stemmers.

O processo de stemming não leva em consideração o significado das palavras, apenas seu formato. 

Em resumo, o stemming é uma técnica de pré-processamento que ajuda a simplificar palavras em um texto, removendo variações gramaticais, para facilitar o processamento e a análise de texto em tarefas de PLN, como classificação de texto, agrupamento, recuperação de informações, entre outras.

## Repesentação tf-idf

O método TF-IDF (Term Frequency-Inverse Document Frequency) é uma técnica amplamente utilizada na análise de texto. Ele atribui importância às palavras com base em quão frequentemente elas aparecem em poucos documentos, associando a essas palavras um valor mais elevado. É uma técnica não supervisionada que não considera diretamente se as palavras são positivas ou negativas. É importante destacar que a conversão de textos para a representação TF-IDF deve ser realizada após a limpeza dos documentos.

O TF-IDF é uma ferramenta valiosa na análise de texto, permitindo identificar as palavras mais importantes em um conjunto de documentos. No contexto da análise de sentimentos em críticas de filmes, por exemplo, o TF-IDF pode ajudar a identificar palavras-chave que são frequentemente associadas a tópicos específicos, mesmo que não estejam diretamente ligadas a críticas positivas ou negativas. Isso pode ser útil para extrair informações relevantes e insights dos documentos de texto.

## N-gramas

Uma das limitações da representação BoW (Bag of Words) é que ela descarta informações sobre a ordem das palavras. Frases como "não é bom, é mau" e "não é mau, é bom" têm a mesma representação, apesar de terem significados opostos. Uma maneira de capturar parte da informação contextual é incluir sequências de duas ou mais palavras que aparecem frequentemente nos documentos na representação BoW. Conjuntos de duas palavras são chamados de bi-gramas, de três palavras são tri-gramas e, em geral, sequências de várias palavras são denominadas n-gramas.

É importante notar que a inclusão de n-gramas pode resultar em um aumento significativo no número de entradas no vocabulário. Em teoria, o número máximo de bi-gramas é igual ao número de palavras individuais no vocabulário elevado ao quadrado, o número máximo de tri-gramas é igual ao número de palavras individuais no vocabulário elevado ao cubo e assim por diante. No entanto, devido à estrutura e características da linguagem escrita, o número de n-gramas é substancialmente menor do que o número máximo teórico, mas ainda assim pode ser muito elevado.

In [ ]:
# Função de tokenização personalizada para excluir palavras com números
def custom_tokenizer(text):
    tokens = re.findall(r'\b\w\w\w+\b', strip_accents_ascii(text.lower()))
    # Excluir palavras com números
    filtered_tokens = [token for token in tokens if not any(char.isdigit() for char in token)]
    return filtered_tokens

In [ ]:
# Registrar o tempo de início
tempo_inicio = time.time()

# Construir um pipeline com vetorização TF-IDF, Regressão Logística com regularização Ridge, stop words, stemming e n-grams
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        stop_words=ENGLISH_STOP_WORDS,
        tokenizer=custom_tokenizer,  # Usar o tokenizador personalizado
        ngram_range=(1, 4),
        min_df=5
    )),
    ('clf', LogisticRegression(penalty='l2', solver='saga', max_iter=1000, C=1, tol=1e-3))
])

# Treinar o modelo
text_clf.fit(X_treino, y_treino)

# Aceder aos nomes das características do TfidfVectorizer
feature_names = text_clf.named_steps['tfidf'].get_feature_names()

# Aceder aos coeficientes do modelo de Regressão Logística
coeficientes = text_clf.named_steps['clf'].coef_[0]

# Criar um dicionário com nomes de recursos e seus coeficientes correspondentes
coef_dict = dict(zip(feature_names, coeficientes))

# Classificar o dicionário por coeficientes em ordem ascendente (menos importante primeiro)
palavras_menos_importantes = sorted(coef_dict.items(), key=lambda x: x[1])[:10]

# Classificar o dicionário por coeficientes em ordem descendente (mais importante primeiro)
palavras_mais_importantes = sorted(coef_dict.items(), key=lambda x: x[1], reverse=True)[:10]

# Imprimir os resultados
print("\nPalavras Mais Positivas:")
for palavra, coef in palavras_menos_importantes:
    print(f"{palavra}: {coef}")

print("\nPalavras Mais Negativas:")
for palavra, coef in palavras_mais_importantes:
    print(f"{palavra}: {coef}")

# Registrar o tempo de término
tempo_fim = time.time()

# Calcular o tempo decorrido
tempo_decorrido = tempo_fim - tempo_inicio

# Imprimir o tempo decorrido em segundos
print(f"\nTempo decorrido: {tempo_decorrido:.2f} segundos")


#print("Vocabulário (Nomes das Features):", feature_names)
#print("Tamanho do Vocabulário (Nomes das Features):", len(feature_names))


Em cima podemos observar as palavras que mais contribuem para uma dada review ter ou não uma boa review. As palavras mais positivas são como é obvio adjetivos e sabores que costumamos considerar agradáveis (como chocolate, uva ou caramelo). O contrário dá-se nas palavras negativas como a cerveja ser considerada aguada, metálica ou de cartão. 

In [ ]:
# Prever rótulos para o conjunto de teste
y_pred = text_clf.predict(X_teste)

# Calcular e imprimir a precisão
precisao = accuracy_score(y_teste, y_pred)
print(f"Precisão: {precisao:.2f}")

# Imprimir a matriz de confusão
matriz_confusao = confusion_matrix(y_teste, y_pred)
print("\nMatriz de Confusão:")
print(matriz_confusao)

# Imprimir o relatório de classificação
relatorio_classificacao = classification_report(y_teste, y_pred)
print("\nRelatório de Classificação:")
print(relatorio_classificacao)


O máximo de precisão que conseguimos foi 29%, quer nós aumentássemos o número de amostras, o número do n-grams ou mudássemos o discriminante logístico por isso conseguimos otimizar a duração do processo a menos de 1 minuto. Acabámos por escolher o Ridge pois este era o que nos dava melhores valores de precisão nas reviews muito boas (score>=9) e muito más (score<=2), ambos os overalls 1 e 10, o min e o max, com 50% o que é o nosso melhor valor. Mudámos também a expressão regular para retirar todas as palavras com menos de 3 letras. Os docentes avisaram que não obter resultados decentes nesta fase é normal por isso não parece existir problema com a nossa abordagem.

# 3. Classificação Binária <a id='section3'></a>

Nesta tarefa, pretende-se saber se o crítico considera a cerveja muito boa ou muito má, baseado no que escreveu. Considere que uma cerveja e considerada muito boa quando obteve uma pontuação global (campo overall) de 9 ou mais valores. Considere ainda que uma cerveja e considerada muito má quando obteve uma pontuação global de 2 ou menos valores.


## 3.1. Treino<a id='section3.1'></a>
<hr style="height:3px"/>

Vamos começar por dividir o conjunto de treino nas reviews muito boas e muito más e descartar as restantes. Vamos usar os mesmos parâmetros que utilizámos na fase anterior mas desta vez utilizamos também o classificador Naive Bayes. O seu princípio básico envolve a aplicação do Teorema de Bayes, assumindo independência condicional entre as características, daí o termo "Naive". É notório pela sua eficácia em problemas de classificação, especialmente em tarefas relacionadas com texto, como análise de sentimentos o que é exatamente o que procuramos neste contexto.

In [ ]:
# Carregar dados do ficheiro pickle
dados_treino = pickle.load(open('rateBeer75Ktrain.p', 'rb'))

# Extrair características (avaliações) e rótulos (classificações gerais)
avaliacoes = [dados_treino[key]['review'] for key in dados_treino]
overalls = [int(re.search(r'\d+', dados_treino[key]['overall']).group()) for key in dados_treino]


# Criar rótulos binários para classificação binária (muito bom ou muito mau)
overall_binarias = [1 if overall >= 9 else 0 if overall <= 2 else None for overall in overalls]


# Remover entradas com None nos rótulos binários
dados = list(zip(avaliacoes, overall_binarias))
dados = [(avaliacao, overall) for avaliacao, overall in dados if overall is not None]
avaliacoes, overall_binarias = zip(*dados)

# Construir pipelines para ambos os classificadores (Regressão Logística e Naive Bayes)
regressao_logistica_clf = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        stop_words=ENGLISH_STOP_WORDS,
        tokenizer=custom_tokenizer,
        ngram_range=(1, 4),
        min_df=5
    )),
    ('clf', LogisticRegression(penalty='l2', solver='saga', max_iter=1000, C=1, tol=1e-3))
])

naive_bayes_clf = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        stop_words=ENGLISH_STOP_WORDS,
        tokenizer=custom_tokenizer,
        ngram_range=(1, 4),
        min_df=5
    )),
    ('clf', MultinomialNB())  # Usar Naive Bayes Multinomial
])

# Lista de classificadores e seus nomes
classificadores = [
    ('Regressão Logística', regressao_logistica_clf),
    ('Naive Bayes', naive_bayes_clf)
]

# Comparar o desempenho de ambos os classificadores
for nome_cls, cls in classificadores:
    print(f"\n*** {nome_cls} ***")
    
    # Dividir dados em conjuntos de treino e teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(avaliacoes, overall_binarias, test_size=0.6, random_state=42)

    # Treinar o modelo
    cls.fit(X_treino, y_treino)

    # Fazer previsões no conjunto de teste
    y_predito = cls.predict(X_teste)

    # Avaliar o modelo
    precisao = accuracy_score(y_teste, y_predito)
    print(f"Precisão: {precisao:.2f}")

    # Relatório de Classificação
    print("\nRelatório de Classificação:")
    print(classification_report(y_teste, y_predito))

    # Matriz de Confusão
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_teste, y_predito))


Como podemos ver pelos resultados em cima ambos os modelos dão nos bastante confiança tendo resultados de 90%. Não foi preciso modicar nada na pipeline apenas mudou o conjunto de teste. Com estas observações podemos usar ambos estes classificadores com todas as certezas.

## 3.2. Teste<a id='section3.2'></a>
<hr style="height:3px"/>
Agora que temos os nossos classificadores escolhidos podemos partir para a fase de teste. Nesta fase vamos usar o ficheiro 'rateBeer25Ktest.p' que é próprio para testes. Começamos por mais uma vez extrair apenas as reviews muito más e muito boas.

In [ ]:
# Carregar dados do ficheiro pickle
dados_teste = pickle.load(open('rateBeer25Ktest.p', 'rb'))

# Extrair características (avaliações) e overall dos dados de teste
avaliacoes_teste = [dados_teste[chave]['review'] for chave in dados_teste]
overalls = [int(re.search(r'\d+', dados_teste[chave]['overall']).group()) for chave in dados_teste]

# Criar rótulos binários para classificação binária (muito bom ou muito mau)
overall_binarios = [1 if overall >= 9 else 0 if overall <= 2 else None for overall in overalls]

# Remover entradas com None nos rótulos binários
dados = list(zip(avaliacoes_teste, overall_binarios, overalls))
dados_filtrados = [(avaliacao, rotulo, pontuacao_real) for avaliacao, rotulo, pontuacao_real in dados if rotulo is not None]
avaliacoes_filtradas, overall_binarios_filtrados, overall_filtrado = zip(*dados_filtrados)

print(f"Tamanho do grupo de teste: {len(overall_binarios_filtrados)}")


Vamos então testar:

In [ ]:
# Definir classificadores
classificadores = [
    ('Regressão Logística', regressao_logistica_clf),
    ('Naive Bayes', naive_bayes_clf)
]

# Testar cada classificador
for nome_clf, clf in classificadores:
    print(f"\n*** {nome_clf} ***")

    # Prever sentimentos para os dados de teste filtrados
    previstos = clf.predict(avaliacoes_filtradas)

    # Converter rótulos previstos em sentimentos legíveis
    pontuacoes_previstas = ["Muito Bom" if pred == 1 else "Muito Mau" for pred in previstos]

    # Avaliar o desempenho do modelo nos dados de teste filtrados
    precisao = accuracy_score(overall_binarios_filtrados, previstos)
    print(f"Precisão: {precisao:.2f}")

    # Métricas adicionais de avaliação
    print("\nRelatório de Classificação:")
    print(classification_report(overall_binarios_filtrados, previstos))

    print("\nMatriz de Confusão:")
    print(confusion_matrix(overall_binarios_filtrados, previstos))

    # Imprimir o sentimento previsto, pontuação geral real e rótulo binário real para cada avaliação nos dados de teste filtrados
    #for i, (sentimento, pontuacao_real) in enumerate(zip(pontuacoes_previstas, overall_filtrado)):
    #    print(f"Avaliação {i + 1}: Sentimento Previsto - {sentimento}, Pontuação Real - {pontuacao_real}")

Como é lógico os resultados são inferiores aos anteriores mas isto não é por muito sendo a maior queda de 3% no classificador Naive Bayes. Com estes testes gerais conseguimos chegar à conclusão que ambos os classificadores estão a funcionar bastante bem mas vamos aprofundar um pouco a questão.

Vamos fazer um pequeno teste numa review fabricada por nós onde vamos utilizar 2 adjetivos negativos e 1 positivo, logo o resultado esperado vai ser uma muito má. Vamos fazer o mesmo para uma muito boa. Acrescentamos também uma média só para testar e observar os resultados.

In [ ]:
# Criar reviews
review_teste_ma = "This beer is bad! The flavor sucks but the aroma is nice."
review_teste_mid = "This beer is alright. The flavor is not good but the aroma smells like caramel."
review_teste_boa = "This beer is great! The flavor sucks but the aroma is nice."

# Iterar sobres as reviews
for review_name, review_text in zip(['Mau', 'Médio', 'Bom'], [review_teste_ma, review_teste_mid, review_teste_boa]):
    print(f"\n*** Review - {review_name} ***")

    # Iterar pelos classificadores
    for clf_name, clf in classificadores:
        print(f"\n** {clf_name} **")

        # Usar o modelo trainado
        pred = clf.predict([review_text])

        #Se 1 muito bom se 0 muito mau
        sentiment = "Muito Bom" if pred[0] == 1 else "Muito Mau"

        print(f"The predicted sentiment for the review is: {sentiment}")


Os resultados dos 2 extremos são os esperados mas existe alguma indecisão no que toca a reviews que não declaram com confiança o que sentem. A diferença nos resultados para a análise do meio  entre os classificadores de Regressão Logística e Naive Bayes pode ser atribuída à forma como cada classificador faz previsões com base nas características fornecidas. 

No caso do classificador Naive Bayes, ele faz previsões usando o teorema de Bayes, assumindo que as características (palavras, neste caso) são condicionalmente independentes dado o rótulo da classe. O Naive Bayes pode ser melhor em capturar a probabilidade de certas combinações de palavras numa análise e não leva em consideração as interações entre as características.


Por outro lado, a Regressão Logística considera a combinação linear das características e aplica uma função logística para fazer previsões. Ela pode capturar relações mais complexas entre características e pode ter um desempenho diferente com base na natureza dos dados.

Se formos analisar os valores dos coeficientes entre os 2 classificadores conseguimos encontrar uma maior diferença entre os mesmos no classificador de Regressão Logística comparando com o Naive Bayes sendo esta a razão provável de neste ser considerado muito mau e no outro o contrário pois naturalmente "not good" vai ter um maior valor comparanando com um cheiro a caramelo.

Concluímos que numa review que explique a sua opinião com adjetivos mais fortes conseguimos decifrar com toda a certeza a sua opinião enquanto que com uma mais no meio seja mais duvidoso.

Por fim antes de continuarmos para o próximo capitulo vamos analisar quais os valores das reviews têm melhor percentagem de resultados corretos.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Categorias de avaliação
categorias = [1, 2, 9, 10]

# Criar gráfico de barras para cada classificador
for nome_clf, clf in classificadores:
    # Dicionário para armazenar os resultados
    resultados = {'total': {categoria: 0 for categoria in categorias}, 'corretos': {categoria: 0 for categoria in categorias}}

    previstos = clf.predict(avaliacoes_filtradas)
    
    # Avaliar o desempenho do modelo nos dados de teste filtrados
    for previsto, real_binario, real_overall in zip(previstos, overall_binarios_filtrados, overall_filtrado):
        resultados['total'][real_overall] += 1
        if previsto == real_binario:
            resultados['corretos'][real_overall] += 1

    # Calcular a percentagem de avaliações corretas para cada categoria
    percentagens_corretas = [resultados['corretos'][categoria] / resultados['total'][categoria] * 100 if resultados['total'][categoria] != 0 else 0 for categoria in categorias]

    # Criar barras com diferentes cores para cada categoria
    cores = plt.cm.rainbow(np.linspace(0, 1, len(categorias)))
    plt.bar(categorias, percentagens_corretas, color=cores, label=f'{nome_clf}')

    # Adicionar rótulos, título e legendas ao gráfico
    plt.xlabel('Overall')
    plt.ylabel('Percentagem de Avaliações Corretas')
    plt.title(f'Desempenho do Classificador "{nome_clf}" por Categoria de Avaliação\n')
    plt.ylim(70, 100)
    plt.show()


Podemos observar pelos 2 gráficos acima que um dos valores é claramente mais dificil de fazer a previsão e este é o 2. Isto deve-se ao facto de os reviewers usarem um palavreado mais rico e neutro quando comparando com os outros valores. Vamos ter como exemplo esta review:

In [ ]:
# Carregar dados do ficheiro pickle
dados_teste = pickle.load(open('rateBeer25Ktest.p', 'rb'))

# Encontrar uma avaliação com uma pontuação global de 2
review_with_score_2 = next((dados_teste[chave] for chave in dados_teste if int(re.search(r'\d+', dados_teste[chave]['overall']).group()) == 2), None)

if review_with_score_2:
    print(f"Avaliação com Pontuação Global de 2:\n{review_with_score_2}")


Como podemos ver este utilizador não utilizou nenhuma das palavras características de uma má review logo fica mais dificil para os classificadores fazerem essa distinção.

Os restantes resultados não oferecem muitas introspeções apenas o facto de o classificador Naive Bayes preferir reviews positivas enquanto que a Regressão Logística prefere negativas. 

# 4. Classificação  Multi-Classe <a id='section4'></a>

Prever a pontuação de três aspetos das críticas (smell, taste e overall). Neste ponto, treine e avalie os classificadores com os dados de treino e verifique se as estimativas do desempenho condizem com os resultados obtidos no conjunto de teste.

## 4.1. Treino<a id='section4.1'></a>
<hr style="height:3px"/>

Vai ter um procedimento exatamente igual à fase anterior só que não vamos ter de filtrar as reviews e vamos usar também o smell e taste. Como não temos muito tempo para realizar este trabalho vamos utilizar o mesmo classificador que viemos a utilizar até agora: a Regressão Logística.

In [ ]:
# Carregar os dados
dados_treino = pickle.load(open('rateBeer75Ktrain.p', 'rb'))

# Extrair características (avaliações) e rótulos (pontuações do aspecto "smell")
reviews = [dados_treino[key]['review'] for key in dados_treino]
smell_labels = [int(re.search(r'\d+', dados_treino[key]['smell']).group()) for key in dados_treino]

# Dividir os dados em conjuntos de treino e teste para "smell"
X_treino, X_teste, smell_y_treino, smell_y_teste = train_test_split(
    reviews, smell_labels, test_size=0.1, random_state=42
)

# Construir um pipeline com vetorização TF-IDF e Logistic Regression (L2 regularization)
smell_clf_lr = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        stop_words=ENGLISH_STOP_WORDS,
        tokenizer=custom_tokenizer,
        ngram_range=(1, 4),
        min_df=5
    )),
    ('clf', LogisticRegression(penalty='l2', solver='saga', max_iter=1000, C=1, tol=1e-3))
])

print("Cheiro\n")

# Treinar o classificador para o aspecto "smell"
smell_clf_lr.fit(X_treino, smell_y_treino)

# Avaliar o classificador nos dados de teste para "smell"
smell_y_pred_lr = smell_clf_lr.predict(X_teste)

# Calcular e imprimir métricas de avaliação para "smell"
accuracy_lr = accuracy_score(smell_y_teste, smell_y_pred_lr)
print(f"Precisao: {accuracy_lr:.2f}")

print("\nRelatório de Classificação:")
print(classification_report(smell_y_teste, smell_y_pred_lr))

print("\Matriz de Confusão:")
print(confusion_matrix(smell_y_teste, smell_y_pred_lr))


In [ ]:
# Extrair características (avaliações) e rótulos (pontuações do aspecto "taste")
taste_labels = [int(re.search(r'\d+', dados_treino[key]['taste']).group()) for key in dados_treino]

# Dividir os dados em conjuntos de treino e teste para "taste"
X_treino_taste, X_teste_taste, taste_y_treino, taste_y_teste = train_test_split(
    reviews, taste_labels, test_size=0.2, random_state=42
)

# Construir um pipeline com vetorização TF-IDF e Logistic Regression (L2 regularization)
taste_clf_lr = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        stop_words=ENGLISH_STOP_WORDS,
        tokenizer=custom_tokenizer,
        ngram_range=(1, 4),
        min_df=5
    )),
    ('clf', LogisticRegression(penalty='l2', solver='saga', max_iter=1000, C=1, tol=1e-3))
])

# Treinar o classificador para o aspecto "taste"
taste_clf_lr.fit(X_treino_taste, taste_y_treino)

# Avaliar o classificador nos dados de teste para "taste"
taste_y_pred_rf = taste_clf_lr.predict(X_teste_taste)

print("Sabor\n")

# Calcular e imprimir métricas de avaliação para "taste"
accuracy_rf_taste = accuracy_score(taste_y_teste, taste_y_pred_rf)
print(f"Precisao: {accuracy_lr:.2f}")

print("\nRelatório de Classificação:\n")
print(classification_report(taste_y_teste, taste_y_pred_rf))

print("Matriz de Confusão:\n")
print(confusion_matrix(taste_y_teste, taste_y_pred_rf))

In [ ]:
dados_treino = pickle.load(open('rateBeer75Ktrain.p', 'rb'))

# Extrair características (avaliações) e rótulos (pontuações do aspecto "smell")
reviews = [dados_treino[key]['review'] for key in dados_treino]

# Extrair características (avaliações) e rótulos (pontuações do aspecto "overall")
overall_labels = [int(re.search(r'\d+', dados_treino[key]['overall']).group()) for key in dados_treino]

# Dividir os dados em conjuntos de treino e teste para "overall"
X_treino_overall, X_teste_overall, overall_y_treino, overall_y_teste = train_test_split(
    reviews, overall_labels, test_size=0.2, random_state=42
)

# Construir um pipeline com vetorização TF-IDF e RandomForestClassifier para "overall"
overall_clf_lr = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        stop_words=ENGLISH_STOP_WORDS,
        tokenizer=custom_tokenizer,  # Usar o tokenizador personalizado
        ngram_range=(1, 4),
        min_df=5
    )),
    ('clf', LogisticRegression(penalty='l2', solver='saga', max_iter=1000, C=1, tol=1e-3))
])

print("Overall\n")

# Treinar o classificador para o aspecto "overall"
overall_clf_lr.fit(X_treino_overall, overall_y_treino)

# Avaliar o classificador nos dados de teste para "overall"
overall_y_pred_rf = overall_clf_lr.predict(X_teste_overall)

# Calcular e imprimir métricas de avaliação para "overall"
accuracy_rf_overall = accuracy_score(overall_y_teste, overall_y_pred_rf)
print(f"Precisao: {accuracy_rf_overall:.2f}")

print("\nRelatório de Classificação:\n")
print(classification_report(overall_y_teste, overall_y_pred_rf))

print("Matriz de Confusão:\n")
print(confusion_matrix(overall_y_teste, overall_y_pred_rf))

## 4.2. Teste<a id='section4.2'></a>
<hr style="height:3px"/>
O processo vai ser, mais uma vez, igual ao anterior.

In [ ]:
# Carregar os dados de teste
dados_teste = pickle.load(open('rateBeer25Ktest.p', 'rb'))

# Extrair características (avaliações) e rótulos para "smell", "taste" e "overall"
reviews_teste = [dados_teste[key]['review'] for key in dados_teste]
smell_labels = [int(re.search(r'\d+', dados_teste[key]['smell']).group()) for key in dados_teste]
taste_labels = [int(re.search(r'\d+', dados_teste[key]['taste']).group()) for key in dados_teste]
overall_labels = [int(re.search(r'\d+', dados_teste[key]['overall']).group()) for key in dados_teste]

# Predictions
smell_y_pred = smell_clf_lr.predict(reviews_teste)
taste_y_pred = taste_clf_lr.predict(reviews_teste)
overall_y_pred = overall_clf_lr.predict(reviews_teste)

# Confusion matrix and classification report for "smell"
accuracy_smell = accuracy_score(smell_labels, smell_y_pred)
print(f"\nPrecisao (Smell): {accuracy_smell:.2f}")
conf_matrix_smell = confusion_matrix(smell_labels, smell_y_pred)
print("\nMatriz de Confusão (Smell):\n")
print(conf_matrix_smell)
print("\nRelatório de Classificação (Smell):\n")
print(classification_report(smell_labels, smell_y_pred))

# Confusion matrix and classification report for "taste"
accuracy_taste = accuracy_score(taste_labels, taste_y_pred)
print(f"\nPrecisao (Taste): {accuracy_taste:.2f}")
conf_matrix_taste = confusion_matrix(taste_labels, taste_y_pred)
print("\nMatriz de Confusão (Taste):\n")
print(conf_matrix_taste)
print("\nRelatório de Classificação (Taste):\n")
print(classification_report(taste_labels, taste_y_pred))

# Confusion matrix and classification report for "overall"
accuracy_overall = accuracy_score(overall_labels, overall_y_pred)
print(f"\nPrecisao (Overall): {accuracy_overall:.2f}")
conf_matrix_overall = confusion_matrix(overall_labels, overall_y_pred)
print("\nMatriz de Confusão (Overall):\n")
print(conf_matrix_overall)
print("\nRelatório de Classificação (Overall):\n")
print(classification_report(overall_labels, overall_y_pred))


Os resultados não são bons mas, mais uma vez, era esperado já que nem os docentes conseguiram bons modelos. No entanto, isto não nos impede de tirar conclusões sobre os mesmos. Vamos criar uma função que nos vai permitir que os modelos tentem advinhar qual o score.

In [ ]:
def predict(review):

    # Cria um dicionario para guardar os valores
    predictions = {}

    # Predict "smell"
    smell_prediction = smell_clf_lr.predict([review])[0]
    predictions['smell'] = int(re.search(r'\d+', str(smell_prediction)).group() if smell_prediction else 0)

    # Predict "taste"
    taste_prediction = taste_clf_lr.predict([review])[0]
    predictions['taste'] = int(re.search(r'\d+', str(taste_prediction)).group() if taste_prediction else 0)

    # Predict "overall"
    overall_prediction = overall_clf_lr.predict([review])[0]
    predictions['overall'] = int(re.search(r'\d+', str(overall_prediction)).group() if overall_prediction else 0)

    return predictions

Criamos também algumas reviews como fizemos anteriormente mas vamos nos focar nos 2 aspetos novos: o cheiro e sabor.

In [ ]:
reviews = [
    "This beer has a bad smelly aroma with hints of citrus but a nice caramel taste.",
    "This beer has a nice chocolate aroma with hints of citrus but a bad watery flavour.",
    "This beer has a bad chocolate aroma with hints of cardboard and a bad flavour.",
    "This beer has a nice chocolate aroma with hints of citrus and great flavour."
]

for i, review in enumerate(reviews, start=1):
    predictions = predict(review)
    print(f"\nPredictions for Review {i}:\n{predictions}")


Como podemos ver os resultados são péssimos bem podemos estar a elogiar ambas as características, como na review 4, que os resultados não passam de medíocres. Também conseguimos observar que, nas 2 primeiras reviews alternámos entre qual a caracteristica elogiavamos, e não mudava absolutamente nada até fez o contrário pois quando dissémos mal do aroma tivémos o resultado melhor do que quando dissemos que era agradável. Isto é claramente culpa da base de dados pois por muitos testes que fizéssemos quer mudássemos o classificador ou as suas características estes foram os melhores resultados.

# 5. PCA <a id='section5'></a>

Infelizmente não tivemos tempo de acabar este tópico mas fica aqui uma breve introdução. A aplicação de PCA (Principal Component Analysis) pode ter impactos diferentes nas tarefas de classificação binária, especialmente quando se lida com dados textuais. 

Sem PCA: A precisão pode ser razoavelmente boa, especialmente se os dados já estiverem num formato que permita um bom desempenho do classificador.

Com PCA: A redução de dimensionalidade pode ajudar a simplificar o modelo, removendo redundâncias nos dados.
Em dados textuais, onde a dimensionalidade pode ser alta, PCA pode ajudar a focar nas principais características, resultando num modelo mais eficiente.

In [ ]:
# Carregar dados do arquivo pickle
dados_treino = pickle.load(open('rateBeer75Ktrain.p', 'rb'))

# Extrair características (avaliações) e rótulos (classificações gerais)
avaliacoes = [dados_treino[key]['review'] for key in dados_treino]
overalls = [int(re.search(r'\d+', dados_treino[key]['overall']).group()) for key in dados_treino]

# Criar rótulos binários para classificação binária (muito bom ou muito mau)
overall_binarias = [1 if overall >= 9 else 0 if overall <= 2 else None for overall in overalls]

# Remover entradas com None nos rótulos binários
dados = list(zip(avaliacoes, overall_binarias))
dados = [(avaliacao, overall) for avaliacao, overall in dados if overall is not None]
avaliacoes, overall_binarias = zip(*dados)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(avaliacoes, overall_binarias, test_size=0.2, random_state=42)

# Função para realizar a classificação com ou sem PCA
def classify_with_pca(X_train, y_train, X_test, y_test, classifier, use_pca=False, n_components=None):
    # Definir o classificador
    classifier.steps[-1] = ('clf', classifier.steps[-1][1].__class__(max_iter=1000, C=1, tol=1e-3))

    # Remover o componente 'svd' se existir
    classifier.steps = [step for step in classifier.steps if step[0] != 'svd']
    
    # Adicionar PCA se necessário
    if use_pca:
        classifier.steps.insert(-1, ('svd', TruncatedSVD(n_components=n_components)))
    
    # Treinar o classificador
    classifier.fit(X_train, y_train)

    # Realizar previsões
    y_pred = classifier.predict(X_test)

    # Avaliar e retornar a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

X_train_array = np.array(X_train)
print(f'Dimensão original do conjunto de dados: {X_train_array.shape[0]}')

# Classificar sem PCA
accuracy_without_pca = classify_with_pca(X_train, y_train, X_test, y_test, regressao_logistica_clf, use_pca=False)
print(f'Precisão sem PCA: {accuracy_without_pca:.2f}')

# Classificar com PCA (determinar o número ótimo de componentes)
best_accuracy = 0
best_n_components = 0

for n_components in range(1, 100, 10):  
    accuracy_with_pca = classify_with_pca(X_train, y_train, X_test, y_test, regressao_logistica_clf, use_pca=True, n_components=n_components)

    if accuracy_with_pca > best_accuracy:
        best_accuracy = accuracy_with_pca
        best_n_components = n_components

print(f'Melhor Precisão com PCA ({best_n_components} componentes): {best_accuracy:.2f}')


Os resultados indicam que o PCA reduziu para 81 componentes  o que fez com que houvesse uma ligeira redução na precisão do modelo. Podemos tirar algumas conclusões:

Sem PCA (Precisão: 0.93): O modelo sem PCA atingiu uma precisão de 93%, o que é bastante bom. Isto sugere que os recursos originais eram informativos e suficientes para obter um bom desempenho.


Com PCA (Melhor Precisão com 81 componentes - 0.91): A aplicação de PCA com uma redução para 81 componentes resultou numa precisão ligeiramente inferior (91%). Isso pode indicar que a informação contida nas primeiras 81 componentes não foi tão discriminativa quanto as características originais.


A aplicação do PCA é um trade-off entre a simplificação do modelo e a preservação da informação. Em suma, apesar da redução na precisão, uma diminuição de 6718 dados para 71 é uma excelente redução e ajuda imenso na simplificação do modelo oferecendo uma maior rapidez. Isto não é necessário nesta classificação binária devida ao já baixo número de características mas pode ser benéfico noutros casos ou até mesmo na nossa classificação multi-classe.

# 6. Conclusões <a id='section6'></a>

Este trabalho prático deu-nos a possibilidade de consolidar todos os campos lecionadas ao longo da disciplina de Aprendizagem Automática como por exemplo: os diferentes tipos de classificação e os seus diversos classificadores, o modelo Bag of Words ou PCA. Apesar de não termos conseguido os melhores resultados acreditamos que conseguimos atingir todos os requisitos demonstrando o domínio que temos sobre a matéria lecionada. Contudo também acreditamos que se tivéssemos mais tempo ou a base de dados fosse melhor escolhida, conseguíamos facilmente atingir resultados mais satisfatórios.